## CS310 Natural Language Processing
## Assignment 5 (part 2): Pretraining BERT with on a Full Dataset

You should re-use the code from A5_bert_toy.ipynb. For clarity, you are suggested to put the code for model definition in a separate file, e.g., model.py, and import it here.

In [1]:
import math
import re
import random
from typing import List, Dict
from pprint import pprint
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

### 1. Data Processing

In [2]:
MAX_LEN = 60 # maximum of width of a batch
MAX_PRED = 10  # maxium number predictions to make in a batch

batch_size = 32
n_layers = 32 # number of Encoder of Encoder Layer
n_heads = 128 # number of heads in Multi-Head Attention
d_model = 768 # Embedding Size
d_ff = 768 * 4  # 4*d_model, FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_segments = 2 # number of segments, ex) sentence A and sentence B


def make_batch(tokens_list: List[int], batch_size: int, word_to_id: Dict):
    batch = []
    positive = negative = 0

    while positive != batch_size/2 or negative != batch_size/2:
        sent_a_index, sent_b_index= random.randrange(len(tokens_list)), random.randrange(len(tokens_list))
        tokens_a, tokens_b= tokens_list[sent_a_index], tokens_list[sent_b_index]

        input_ids = [word_to_id['[CLS]']] + tokens_a + [word_to_id['[SEP]']] + tokens_b + [word_to_id['[SEP]']]
        segment_ids = [1] * (1 + len(tokens_a) + 1) + [2] * (len(tokens_b) + 1)

        # The following code is used for the Masked Language Modeling (MLM) task.
        n_pred =  min(MAX_PRED, max(1, int(round(len(input_ids) * 0.15)))) # Predict at most 15 % of tokens in one sentence
        masked_candidates_pos = [i for i, token in enumerate(input_ids)
                          if token != word_to_id['[CLS]'] and token != word_to_id['[SEP]']]
        random.shuffle(masked_candidates_pos)
        masked_tokens, masked_pos = [], []
        for pos in masked_candidates_pos[:n_pred]:
            masked_pos.append(pos)
            masked_tokens.append(input_ids[pos])
            ### START YOUR CODE ###
            # Throw a dice to decide if you want to replace the token with [MASK], random word, or remain the same
            # pass
            dice = random.random()
            if dice < 0.8:
                input_ids[pos] = word_to_id['[MASK]']
            else:
                dice = random.random()
                if dice<0.5:
                    input_ids[pos] = random.randint(0, VOCAB_SIZE-1)
            ### END YOUR CODE ###

        # Make zero paddings
        n_pad = MAX_LEN - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        # Zero padding (100% - 15%) of thetokens
        if MAX_PRED > n_pred:
            n_pad = MAX_PRED - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        # The following code is used for the Next Sentence Prediction (NSP) task.
        ### START YOUR CODE ###
        # Decide if the is_next label is positive or negative, by comparing sent_a_index and sent_b_index
        # Don't forget to increment the positive/negative count
        # pass
        if sent_a_index + 1 == sent_b_index:  # Decide if the is_next label is positive or negative, by comparing sent_a_index and sent_b_index
            is_next = 1  # Positive example
            positive += 1
        else:
            is_next = 0  # Negative example
            negative += 1

        batch.append((input_ids, segment_ids, masked_tokens, masked_pos, is_next))

        if positive+negative==batch_size:
            break

        ### END YOUR CODE ###

    return batch


def get_pad_attn_mask(seq_q, seq_k):
    batch_size, seq_len = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=seq_len), one is masking
    return pad_attn_mask.expand(batch_size, seq_len, len_k)  # batch_size x seq_len x len_k





train_file = "train.txt"
test_file = "test.raw.txt"

# 读取train.txt文件
with open(train_file, "r", encoding="utf-8") as f:
    train_text = f.read()

# 读取test.raw.txt文件
with open(test_file, "r", encoding="utf-8") as f:
    test_text = f.read()

# 合并train_text和test_text
all_text = train_text + test_text



sentences = re.sub("[。，！？；\\-]", '', all_text.lower()).split('\n')  # filter '.', ',', '?', '!'

word_types = sorted(list(set(" ".join(sentences))))


# Add the special tokens to the vocabulary
word_to_id = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}
for i, w in enumerate(word_types):
    word_to_id[w] = i + 4
id_to_word = {i: w for i, w in enumerate(word_to_id)}
VOCAB_SIZE = len(word_to_id)
print(VOCAB_SIZE)

tokens_list = []
for sentence in sentences:
    tokens = [word_to_id[s] for s in list(sentence)]
    tokens_list.append(tokens)

train_sentences = re.sub("[。，！？；\\-]", '', train_text.lower()).split('\n')  # filter '.', ',', '?', '!'
train_token_list=[]
for sentence in train_sentences:
    tokens = [word_to_id[s] for s in list(sentence)]
    train_token_list.append(tokens)

test_sentences = re.sub("[。，！？；\\-]", '', test_text.lower()).split('\n')  # filter '.', ',', '?', '!'
test_token_list=[]
for sentence in test_sentences:
    tokens = [word_to_id[s] for s in list(sentence)]
    test_token_list.append(tokens)

# Test
random.seed(0)
batch = make_batch(tokens_list, batch_size, word_to_id)
input_ids, segment_ids, masked_tokens, masked_pos, is_next = map(torch.LongTensor, zip(*batch))

sample = 2

print('sampled text:')
print([id_to_word[w.item()] for w in input_ids[sample] if id_to_word[w.item()] != '[PAD]'])
print()
print('input_ids:', input_ids[sample])
print('segment_ids:', segment_ids[sample])
print('masked_tokens:', masked_tokens[sample])
print('masked_pos:', masked_pos[sample])
print('is_next:', is_next[sample].item())



1525
sampled text:
['[CLS]', '瞻', '[MASK]', '阕', '者', '虚', '室', '[MASK]', '白', '吉', '祥', '止', '[MASK]', '[SEP]', '以', '[MASK]', '饮', '酒', '者', '始', '乎', '治', '常', '卒', '乎', '[MASK]', '泰', '至', '则', '多', '奇', '乐', '[SEP]']

input_ids: tensor([   1,  977,    3, 1417, 1094, 1200,  403,    3,  955,  251, 1000,  774,
           3,    2,   74,    3, 1469, 1386, 1094,  367,   39,  812,  467,  212,
          39,    3,  821, 1138,  176,  343,  354,   40,    2,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0])
segment_ids: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
masked_tokens: tensor([508, 926, 994,  46, 774,   0,   0,   0,   0,   0])
masked_pos: tensor([ 2,  7, 15, 25, 12,  0,  0,  0,  0,  0])
is_next: 0


### 2. Model Training

In [3]:
class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(VOCAB_SIZE, d_model, padding_idx=0)  # token embedding
        self.pos_embed = nn.Embedding(MAX_LEN, d_model)  # position embedding
        self.seg_embed = nn.Embedding(n_segments + 1, d_model, padding_idx=0)  # segment(token type) embedding
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, seg):
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long)
        pos = pos.unsqueeze(0).expand_as(x)  # (seq_len,) -> (batch_size, seq_len)
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size x n_heads x seq_len x =seq_len]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn

class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)
        self.output_linear=nn.Linear(n_heads * d_v, d_model)
        self.layerNorm=nn.LayerNorm(d_model)

    def forward(self, x, attn_mask):
        # x: [batch_size x seq_len x d_model]
        residual, batch_size = x, x.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(x).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size x n_heads x seq_len x d_k]
        k_s = self.W_K(x).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size x n_heads x seq_len x d_k]
        v_s = self.W_V(x).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size x n_heads x seq_len x d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size x n_heads x seq_len x seq_len]

        # context: [batch_size x n_heads x seq_len x d_v], attn: [batch_size x n_heads x seq_len x seq_len]
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v) # context: [batch_size x seq_len x n_heads * d_v]
        output = nn.Linear(n_heads * d_v, d_model)(context)
        output=self.output_linear(context)
        # return nn.LayerNorm(d_model)(output + residual), attn # output: [batch_size x seq_len x d_model]
        return self.layerNorm(output + residual), attn

# Activation function: GELU (Gaussian Error Linear Unit)
def gelu(x):
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # (batch_size, seq_len, d_model) -> (batch_size, seq_len, d_ff) -> (batch_size, seq_len, d_model)
        return self.fc2(gelu(self.fc1(x)))

class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        ### START YOUR CODE ###
        # enc_outputs, attn = None, None
        enc_outputs, attn = self.enc_self_attn( enc_inputs, enc_self_attn_mask)
        enc_outputs = self.pos_ffn(enc_outputs)
        ### END YOUR CODE ###
        return enc_outputs, attn

class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.embedding = Embedding()
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

         # for NSP task
        self.fc1 = nn.Linear(d_model, d_model)
        self.activ1 = nn.Tanh()
        self.classifier = nn.Linear(d_model, 2)

        # for MLM task
        self.fc2 = nn.Linear(d_model, d_model)
        self.activ2 = gelu
        self.norm = nn.LayerNorm(d_model)
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight # decoder is shared with embedding layer
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_pad_attn_mask(input_ids, input_ids)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
            # output : [batch_size, len, d_model], attn : [batch_size, n_heads, d_mode, d_model]

        # Use the representation of [CLS] to produce logits for NSP task
        ### START YOUR CODE ###
        # logits_clsf = None
        cls_output = output[:, 0]  # [CLS] token representation
        cls_output = self.activ1(self.fc1(cls_output))
        logits_clsf = self.classifier(cls_output)
        ### END YOUR CODE ###

        # Gather the representations of masked tokens to produce logits for MLM task
        ### START YOUR CODE ###
        # logits_lm = None
        masked_output = torch.gather(output, 1, masked_pos.unsqueeze(-1).expand(-1, -1, output.size(-1)))
        masked_output = self.activ2(self.fc2(masked_output))
        masked_output = self.norm(masked_output)
        logits_lm = self.decoder(masked_output) + self.decoder_bias
        ### END YOUR CODE ###

        return logits_lm, logits_clsf


In [4]:
random.seed(0)
torch.manual_seed(0)

model = BERT()
criterion = nn.CrossEntropyLoss() # You can also try two separate losses for each task
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

batch = make_batch(train_token_list, batch_size, word_to_id)
input_ids, segment_ids, masked_tokens, masked_pos, is_next = map(torch.LongTensor, zip(*batch))


for epoch in range(500):
    optimizer.zero_grad()

    ### START YOUR CODE ###
    # logits_lm, logits_clsf = None, None
    # Hint: Check the shape of logits_lm and decide if post-processing is needed
    # loss = None
    logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)

    # Reshape logits_lm if needed
    if logits_lm.dim() == 3:
        logits_lm = logits_lm.view(-1, logits_lm.size(-1))

    loss_lm = criterion(logits_lm, masked_tokens.view(-1))
    loss_clsf = criterion(logits_clsf, is_next)
    loss = loss_lm + loss_clsf
    ### END YOUR CODE ###

    if (epoch + 1) % 5 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

Epoch: 0005 cost = 75.624565
Epoch: 0010 cost = 63.398083
Epoch: 0015 cost = 53.859783
Epoch: 0020 cost = 45.883751
Epoch: 0025 cost = 37.718666
Epoch: 0030 cost = 31.988184
Epoch: 0035 cost = 26.031408
Epoch: 0040 cost = 20.947020
Epoch: 0045 cost = 16.453720
Epoch: 0050 cost = 12.353612
Epoch: 0055 cost = 8.769114
Epoch: 0060 cost = 5.724162
Epoch: 0065 cost = 5.010586
Epoch: 0070 cost = 4.841403
Epoch: 0075 cost = 4.517253
Epoch: 0080 cost = 4.119666
Epoch: 0085 cost = 3.845573
Epoch: 0090 cost = 3.639975
Epoch: 0095 cost = 3.501805
Epoch: 0100 cost = 3.393349
Epoch: 0105 cost = 3.310125
Epoch: 0110 cost = 3.229298
Epoch: 0115 cost = 3.173988
Epoch: 0120 cost = 3.116209
Epoch: 0125 cost = 3.070920
Epoch: 0130 cost = 3.040479
Epoch: 0135 cost = 3.001628
Epoch: 0140 cost = 2.970319
Epoch: 0145 cost = 2.937093
Epoch: 0150 cost = 2.909990
Epoch: 0155 cost = 2.905712
Epoch: 0160 cost = 2.894790
Epoch: 0165 cost = 2.885569
Epoch: 0170 cost = 2.878906
Epoch: 0175 cost = 2.879819
Epoch: 018

### 3. Evaluation

In [5]:
# Evaluation
model.eval()
total_mlm_correct = 0
total_nsp_correct = 0
total_samples = 0


batch = make_batch(test_token_list, batch_size, word_to_id)
test_input_ids, test_segment_ids, test_masked_tokens, test_masked_pos, test_is_next = map(torch.LongTensor, zip(*batch))

with torch.no_grad():
    test_logits_lm, test_logits_clsf = model(test_input_ids, test_segment_ids, test_masked_pos)

    # Reshape test_logits_lm if needed
    if test_logits_lm.dim() == 3:
        test_logits_lm = test_logits_lm.view(-1, test_logits_lm.size(-1))

    test_loss_lm = criterion(test_logits_lm, test_masked_tokens.view(-1))
    test_loss_clsf = criterion(test_logits_clsf, test_is_next)
    test_loss = test_loss_lm + test_loss_clsf

    test_predicted_lm = test_logits_lm.argmax(dim=-1)
    test_accuracy_lm = (test_predicted_lm == test_masked_tokens.view(-1)).float().mean().item()

    test_predicted_clsf = test_logits_clsf.argmax(dim=-1)
    test_accuracy_clsf = (test_predicted_clsf == test_is_next).float().mean().item()

# print('Test Loss:', '{:.6f}'.format(test_loss))
print('MLM Accuracy:', '{:.2%}'.format(test_accuracy_lm))
print('NSP Accuracy:', '{:.2%}'.format(test_accuracy_clsf))

Test Loss: 12.158834
MLM Accuracy: 59.06%
NSP Accuracy: 100.00%
